<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/DLScratch1Study/DLScratch1_Ch7_Prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 밑바닥부터 시작하는 딥러닝 7장, 8장 - CNN

합성곱 신경망의 기본 내용 및 구현에 대해 알아보자.
8장은 발전방향 관련 내용이므로, 통합해서 정리하는게 낫지 않을까 싶다.

## CNN의 전체 구조 및 용어
CNN에서는
- convolution layer(합성곱 계층)
- pooling layer(풀링 계층)
이 새로 등장한다.

### 합성곱 계층
앞에서 보았던 Affine 계층 같은 경우에는 모든 인접하는 계층의 뉴런이 서로 연결되어 있는 상태이다. 이 경우, 데이터의 형상이 무시된다.

**데이터의 형상을 유지하는 전달법이 바로 CNN**이다.

CNN에서는 입출력 데이터를 feature map(특징 맵) 이라고 부른다.

####합성 곱 연산
합성 곱 연산은 이미지 처리에서의 필터 연산으로, 

**입력 데이터  필터 -> 출력 데이터 **

의 흐름을 따라가게 된다. 필터를 일정 간격으로 이동하여, 입력데이터에 내적과 동일한 연산 방식을 적용한다.

(실전에서는 여기에 편향을 더하기도 하다)

#### 패딩(padding)
패딩은 입력 데이터 주변을 0으로 채우는 행위이다. 폭의 값에 따라 채워지는 0의 두께가 결정된다.

패딩을 통해서 출력 크기를 조정할 수 있다. 패딩이 없을 경우 합성 곱 연산을 통해 출력층의 크기가 계속 줄어들어 더 이상 합성 곱 연산을 할 수 없을 때가 생기게 된다. 이를 막기 위해 패딩을 적용한다.

#### 스트라이드(stride)
보폭이라는 뜻으로, 스트라이드 값 만큼 필터를 이동하게 된다.
즉, 이 경우 출력은 작아지게 된다.

입력크기, 필터크기, 출력크기, 패딩, 스프라이드에 대한 관계식은 다음과 같다.

- 입력크기 (H, W)
- 필터크기 (FH, FW)
- 출력크기 (OH, OW)
- 패딩 P
- 스트라이드 S

$$ OH = \frac{H+2P-FH}{S}+1$$

$$ OW = \frac{W+2P-FW}{S}+1$$


### 3차원 데이터의 합성곱 연산 및 정의

- 입력 데이터가 길이 방향으로 특징 맵이 확장된다.
- 채널마다 수행해서 결과를 더한다.

이를 고려하기 위해 블록으로 생각해본다.

채널수 $C$ * 높이 $H$ * 너비 $W$ 를 필터로 잘라 출력 데이터를 만든다.

여기서 필터 또한, 이 경우에는 채널수가 $C$가 된다. 이 경우, 출력 데이터는 한 장이 된다. 따라서 출력데이터를 다수 채널로 만들기 위해서는 필터의 개수를 늘려야 하며 이 필터의 개수를 $FN$으로 표현하려 한다.

따라서, 최종 얻게되는 출력 데이터의 형상은 $(FN, OH, OW)$가 된다.

### Batch 처리
배치 처리의 경우에는 데이터 개수를 정해야 하므로 이젠 3차원이 아니라 4차원이 된다.

- 입력데이터 : (N(데이터 수), C(채널), H, W)
- 필터 : (FN(필터개수), C(채널), FH, FW)
- 출력데이터 : (N(데이터 수), FN(필터개수 만큼의 채널), OH, OW)
- 편향 : (FN(필터개수), 1, 1)

## Pooling 계층
풀링은 가로/세로 방향의 공간을 줄인다.
영역의 원소를 하나로 집약하여 공간 크기를 줄인다.

- Max Pooling : M * M 대상 원소 중 하나를 꺼낸다.
- Average Pooling : M * M 대상 원소 중 평균을 꺼낸다.

풀링 계층은 다음과 같은 특징을 가진다.
- 학습해야 할 매개변수가 없다.
- 채널 수가 변하지 않는다. (채널마다 독립적으로 계산함)
- 입력의 변화에 영향을 적게 받는다.

## 합성곱 / 풀링 계층의 구현

In [3]:
# 4차원 배열 (데이터개수, 채널, 높이, 너비)
import numpy as np

x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

for으로 4차원을 구현한다? 매우 유감!
이 책에서는 im2col이라는 편의함수를 사용한다.

im2col의 주요 역할은  **4차원 데이터를 2차원으로 변환**한다.


In [4]:
# im2col 구성들.
def smooth_curve(x):
    """손실 함수의 그래프를 매끄럽게 하기 위해 사용
    
    참고：http://glowingpython.blogspot.jp/2012/02/convolution-with-numpy.html
    """
    window_len = 11
    s = np.r_[x[window_len-1:0:-1], x, x[-1:-window_len:-1]]
    w = np.kaiser(window_len, 2)
    y = np.convolve(w/w.sum(), s, mode='valid')
    return y[5:len(y)-5]


def shuffle_dataset(x, t):
    """데이터셋을 뒤섞는다.

    Parameters
    ----------
    x : 훈련 데이터
    t : 정답 레이블
    
    Returns
    -------
    x, t : 뒤섞은 훈련 데이터와 정답 레이블
    """
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation,:] if x.ndim == 2 else x[permutation,:,:,:]
    t = t[permutation]

    return x, t

def conv_output_size(input_size, filter_size, stride=1, pad=0):
    return (input_size + 2*pad - filter_size) / stride + 1


def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]


이제 합성곱 계층을 구현하자.

im2col(input_data, filter_h, filter_w, stride=1, pad=0)
구조로 간다.
input data는 위에서 본 4차원 배열로 이루어진다.

In [7]:
x1 = np.random.rand(1, 3, 7, 7)  # 7 * 7 형태로 3채널로 이루어진 데이터 셋 1개 추출.
col1 = im2col(x1, 5, 5, stride=1, pad = 0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7) # 이번에는 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad = 0)
print(col2.shape)

(9, 75)
(90, 75)


In [ ]:
class Convolution:
  def __init__(self, W, b, stride = 1, pad = 0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_w = self.W.reshape(FN, -1).T
    # 이렇게 하면 앞의 4차원 배열을 FN * 적당히로 묶어서 동일 원소개수로...
    out = np.dot(col, col_W) + self.b

    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    # N, H, W, C를  N, C, H, W 순서로 변경.

    return out

Pooling 또한 가능하다.
1. 입력 데이터를 전개한다.
2. 행별 최댓값을 구한다.
3. 적절한 모양으로 성형한다.

In [ ]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)

    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h * self.pool_w)

    out = np.max(col, axis = 1)

    out = out.reshae(N, out_h, out_w, C).transpose(0, 3, 1, 2)

    return out_h

## CNN의 구현